# Chapter 12. 파이썬 모델링 라이브러리

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 저자 깃허브 주소
path_ex = 'https://raw.githubusercontent.com/wesm/pydata-book/refs/heads/3rd-edition/examples/'
path_da = 'https://raw.githubusercontent.com/wesm/pydata-book/refs/heads/3rd-edition/datasets/'

In [3]:
# 구글 드라이브를 코랩에 연결하여 데이터 로딩
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 구글 드라이브 주소(examples)
path_gg_ex = '/content/drive/MyDrive/Colab Notebooks/연희전문학원/Python for DA/examples/'

## 12.1 판다스와 모델 코드의 인터페이스
- 피처 엔지니어링(feature engineering): 모델 개발 과정에서 중요한 단계로, 특징을 선택하고 추출하는 것
- 원시 데이터셋으로부터 모델링에서 유용할 만한 정보를 추출하는 변환이나 분석 과정
- 데이터 요약, groupby 도구 등이 자주 사용됨.
- 판다스와 다른 분석 라이브러리는 주로 넘파이 배열을 사용해 연계
- to_numpy 메서드: DataFrame을 넘파이 배열로 변환
- to_numpy 메서드는 데이터가 한 가지 타입으로 이루어져 있다는 가정하에 사용 (아닐 경우 ndarray 반환)

In [5]:
data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [6]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [7]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [8]:
# numpy 배열 DataFrame으로 되돌리기
df2 = pd.DataFrame(data.to_numpy(), columns=["one", "two", "three"])

df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [9]:
df3 = data.copy()

df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [10]:
df3.to_numpy() # 데이터가 한 가지 타입이 아닐 경우 ndarray 반환

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [11]:
# 전체 열 중 일부만 사용하고 싶은 경우
model_cols = ['x0', 'x1']

data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [12]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])

data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [13]:
# 'category' 열 더미 값으로 치환하기
dummies = pd.get_dummies(data.category, prefix='category', dtype=float) # prefix: 접두사

# 'category' 열 삭제 후 더미와 조인
data_with_dummies = data.drop('category', axis=1).join(dummies)

data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1.0,0.0
1,2,-0.01,0.0,0.0,1.0
2,3,0.25,3.6,1.0,0.0
3,4,-4.10,1.3,1.0,0.0
4,5,0.00,-2.0,0.0,1.0


## 12.2 patsy로 모델 생성하기
- patsy: 통계 모델(특히 선형 모델)을 위한 파이썬 라이브러리
- R이나 S 통계 프로그래밍 언어에서 사용하는 수식 문법과 유사한 형식의 문자열 기반 수식 문법 제공
- statsmodels 설치 시 patsy 자동으로 설치
- 수식 문법은 특수한 형태의 문자열 (예. y ~ x0 + x1)

In [ ]:
# ! pip install statsmodels

In [16]:
# a + b 문법: 모델을 위해 생성된 배열 설계
# patsy.dmatrices 함수: 수식 문자열과 데이터셋을 함께 받아 선형 모델을 위한 설계 배열 생성
data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

import patsy

y, X = patsy.dmatrices('y ~ x0 + x1', data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [17]:
X # Intercept: 최소제곱법 회귀와 같은 선형 모델을 위한 표현

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [20]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [21]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [23]:
# 모델에 0을 더해 Intercept(절편) 제거
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [24]:
# patsy 객체는 최소제곱법 회귀 분석을 위해 numpy.linalg.lstsq 같은 알고리즘에 바로 넘길 수 있음
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

In [25]:
# 모델 메타데이터는 design_info 속성을 통해 얻을 수 있음
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [26]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names) # squeeze 메서드는 차원을 축소(압축)하는 메서드
coef

,0
Intercept,0.312910
x0,-0.079106
x1,-0.265464


### 12.2.1 patsy 용법으로 데이터 변환하기
- 파이썬 코드를 patsy 용법과 섞어서 사용 가능
- 자주 사용하는 변수 변환으로는 표준화 (평균 0 분산1)와 중심화(평균값 뺌) : pasty에는 내장 함수 존재
- 중심화나 표준화 같은 변환을 적용(상태를 가지는 변환) 시 새로운 데이터 기반으로 예측하는 용도로 모델을 사용한다면 새로운 데이터셋 변경 시 원본 데이터의 표준편차나 평균 같은 통계를 사용하므로 주의
- patsy.build_design_matrices 함수: 입력으로 사용되는 원본 데이터셋에서 저장한 정보를 사용해 출력 데이터에 변환 적용

In [27]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [28]:
# patsy.build_design_matrices 함수: 입력으로 사용되는 원본 데이터셋에서 저장한 정보를 사용해 출력 데이터에 변환 적용
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})

new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  x0  np.log(np.abs(x1) + 1)
           1   6                 1.41099
           1   7                 0.40547
           1   8                 0.00000
           1   9                 1.19392
   Terms:
     'Intercept' (column 0)
     'x0' (column 1)
     'np.log(np.abs(x1) + 1)' (column 2)]

### 12.2.2 범주형 데이터와 patsy
- patsy에서 숫자가 아닌 용법을 사용하면 기본적으로 더미 변수로 변환됨
- 만약 Intercept가 존재한다면 공선성을 피하기 위해 레벨 중 하나는 남겨두게 됨.
- 공선성: 독립 변수들 간의 강한 상관관계로 인해 회귀모델이 불안정해지는 문제
- Intercept: 좌표평면상의 그래프가 x축과 y축과 만나는 점의 좌표
- patsy는 특정 순서에 따라 데이터를 변환하는 방법과 범주형 데이터를 변환하는 여러 방법 제공

In [29]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [30]:
# Intercept 생량: 각 범줏값의 열은 모델 설계 배열에 포함
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [32]:
# 숫자 열은 C 함수를 이용해서 범주형으로 해석 가능
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [34]:
# 모델에서 여러 범주형 항을 사용하는 경우
# 분산분석 모델에서처럼 key1:key2 같은 용법을 사용할 수 있게 되므로 더 복잡
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [35]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [36]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## 12.3 statsmodels 소개
- statsmodels: 다양한 종류의 통계 모델 피팅, 통계 테스트 수행, 데이터 탐색, 시각화를 위한 파이썬 라이브러리
- statsmodels는 더 고전적인 빈도주의적 통계 메서드를 포함하지만 베이지안 메서드나 머신러닝 모델을 다른 라이브러리에서 찾을 수 있음
- 일반적으로 누락된 데이터를 처리하지 못함

[statsmodels가 포함하는 모델]
- 선형 모델, 일반 선형 모델, 로버스트 선형 모델
- 선형 혼합 효과 모델
- 분산분석 (ANOVA) 메서드
- 시계열 처리 및 상태 공간 모델
- 일반적률추정법(Generalized Method of Moments, GMM)

### 12.3.1 선형 모델 예측 하기
- statsmodels에는 기본적인 선형 회귀 (예. 최소제곱법) 부터 더 복잡한 선형 회귀(예. 반복 재가중 최소제곱법(IRLS)) 까지 존재
- statsmodels 선형 모델은 두 가지 주요한 인터페이스로 배열 기반과 용법 기반이 있음
- sm.add_constant 함수로:기존 행렬에 Intercept 열 추가
- sm.OLS 클래스: 최소제곱법 선형 회귀에 피팅
- fit 메서드: 에측 모델 매개변수와 다른 분석 정보를 포함하는 회귀 결과 객체 반환
- summary 메서드 호출 : 해당 모델의 자세한 분석 결과 출력
- params: 회귀 모델의 파라미터(회귀 계수)를 반환
- tvalues: 선형 회귀 모델의 각 변수에 대한 t-값을 반환
t-값은 회귀 계수가 0이라는 귀무 가설(null hypothesis)을 검정하는 데 사용.
높은 t-값은 독립 변수가 종속 변수에 유의미한 영향을 미친다는 것을 의미
- predict: 주어진 새로운 샘플 데이터를 통해 예측 모델 매개변수에 전달한 예측값 계산

In [37]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [38]:
# 무작위 데이터로 선형 모델 생성하기

# 동일한 난수 발생을 위해 시드 값 고정
rng = np.random.default_rng(seed=12345)

# 특정 평균과 분산을 가지는 정규분포 데이터를 생성하는 헬퍼 함수 정의
def dnorm(mean, variance, size=1): # 매개변수: 평균, 분산, 크기
  if isinstance(size, int):
    size = size,
# 평균 + 분산의 제곱근 * 사이즈를 곱한만큼의 표준정규분포 (Standard normal distribution)로부터 샘플링된 난수를 반환
  return mean + np.sqrt(variance) * rng.standard_normal(*size)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)
          ]

# eps: epsilon 오차
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [39]:
X[:5]

array([[-0.90050602, -0.18942958, -1.0278702 ],
       [ 0.79925205, -1.54598388, -0.32739708],
       [-0.55065483, -0.12025429,  0.32935899],
       [-0.16391555,  0.82403985,  0.20827485],
       [-0.04765129, -0.21314698, -0.04824364]])

In [40]:
y[:5]

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445])

In [42]:
# 선형 모델은 일반적으로 Intercept와 함께 피팅

# sm.add_constant 함수로 기존 행렬에 Intercept 열 추가
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

In [43]:
# sm.OLS 클래스 : 최소제곱법 선형 회귀에 피팅
model = sm.OLS(y, X)

# fit 메서드: 에측 모델 매개변수와 다른 분석 정보를 포함하는 회귀 결과 객체 반환
results = model.fit()
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [44]:
# results의 summary 메서드 호출 : 해당 모델의 자세한 분석 결과 출력
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Wed, 05 Feb 2025   Prob (F-statistic):                    2.66e-13
Time:                        12:54:32   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [45]:
# 모델 매개변수가 들어간 DataFrame
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

data['y'] = y

data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [46]:
# statsmodels의 API와 patsy 문자열 용법 사용
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

# 적합된 회귀 모델의 파라미터(회귀 계수)를 반환
results.params

"""
Intercept (계수): 절편
col0의 계수: col0가 y에 미치는 영향
col1의 계수: col1가 y에 미치는 영향
col2의 계수: col2가 y에 미치는 영향
"""

,0
Intercept,-0.020799
col0,0.065813
col1,0.268970
col2,0.449419


In [47]:
# 적합된 선형 회귀 모델의 각 변수에 대한 t-값을 반환
#  t-값은 회귀 계수가 0이라는 귀무 가설(null hypothesis)을 검정하는 데 사용.
# 높은 t-값은 독립 변수가 종속 변수에 유의미한 영향을 미친다는 것을 의미
results.tvalues

,0
Intercept,-0.652501
col0,1.219768
col1,6.312369
col2,6.567428


In [48]:
# predict: 주어진 새로운 샘플 데이터를 통해 예측 모델 매개변수에 전달한 예측값 계산
results.predict(data[:5])

,0
0,-0.592959
1,-0.531160
2,0.058636
3,0.283658
4,-0.102947


### 12.3.2 시계열 처리 예측하기
- statsmodels에 포함된 또 다른 모델 클래스로 시계열 분석을 위한 모델이 있음

[시계열 분석 모델]
- 자기 회귀 (autoregressive, AR) 처리
- 칼만 필터
- 다른 상태 공간 모델
- 다변 자기 회귀 모델

In [49]:
# 자기 회귀 구조와 노이즈를 이용한 시계열 데이터 실험
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
  new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
  values.append(new_x)

In [50]:
from statsmodels.tsa.ar_model import AutoReg

MAXLAGS = 5

model = AutoReg(values, MAXLAGS)

results = model.fit()

In [51]:
# 예측된 매개변수 확인
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])

## 12.4 사이킷런 소개
- 사이킷런: 범용 파이썬 머신러닝 툴킷
- 표준적인 지도 학습과 비지도 학습 메서드를 포함하고 있으며 모델 선택, 평가, 데이터 변형, 데이터 적재, 모델 유지 및 기타 작업을 위한 도구 제공
- 분류, 클러스터링(군집), 예측, 여러 작업에 사용 가능
- 일반적으로 누락된 데이터를 처리하지 못함

In [52]:
# ! pip install scikit-learn

In [53]:
# 타이타닉 생존자 데이터셋 불러오기

# 훈련 데이터
train = pd.read_csv(path_da + '/titanic/train.csv')

# 테스트 데이터
test = pd.read_csv(path_da + '/titanic/test.csv')

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
# 누락된 값 있는지 확인
train.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [55]:
test.isna().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [56]:
# 나이(Age) 결측치 보완하기 (학습 데이터셋의 중간값 채워넣기)

# 학습 데이터셋 나이 컬럼의 중간값 구하기
impute_value = train['Age'].median()

# 결측치 보완하기
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [57]:
# Sex 열 인코딩 (IsFemale 열 추가)
train['IsFemale'] = (train['Sex'] == 'femail').astype(int)

test['IsFemale'] = (test['Sex'] == 'femail').astype(int)

In [58]:
# 모델 변수 선언 및 넘파이 배열 생성

predictors = ['Pclass', 'IsFemale', 'Age']

X_train = train[predictors].to_numpy()

X_test = test[predictors].to_numpy()

y_train = train['Survived'].to_numpy()

X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  0., 38.],
       [ 3.,  0., 26.],
       [ 1.,  0., 35.],
       [ 3.,  0., 35.]])

In [59]:
y_train[:5]

array([0, 1, 1, 1, 0])

In [61]:
# 사이킷런의 LogisticRegression 모델 이용해서 model 인스턴스 생성
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

# model의 fit 메서드 이용해서 학습 데이터에 피팅
model.fit(X_train, y_train)

# model.predict 이용해서 테스트 데이터셋으로 예측
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### 교차 검증 학습
- 학습 데이터를 분할해 예측을 위한 샘플로 활용하는 방식으로 작동
- 평균 제곱 오차(MSE)와 같은 모델 정확도 점수에 기반해 모델 매개변수에 대한 그리드 검색 수행
- 로지스틱 회귀 같은 모델은 교차 검증을 내장한 추정 클래스 제공
- LogisticRegressionCV 클래스는 모델 정규화 매개변수 C에 대한 그리드 검색을 얼마나 정밀하게 수행할 것인지 나타내는 매개변수와 함께 사용 가능
- 교차 검증된 모델은 학습 시간이 오래 걸리나 더 나은 성능을 보여주기도 함

In [63]:
from sklearn.linear_model import LogisticRegressionCV

model_cv = LogisticRegressionCV(Cs=10)

model_cv.fit(X_train, y_train)
LogisticRegressionCV()

LogisticRegressionCV()

In [65]:
# 직접 교차 검증 수행 시 cross_val_score 헬퍼 함수 사용
# cross_val_score 함수: 데이터를 분할하는 과정 도와줌

from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)

scores = cross_val_score(model, X_train, y_train, cv=4) # 학습 데이터를 겹치지 않는 4개의 그룹으로 나눔

scores

array([0.632287  , 0.69955157, 0.72197309, 0.72072072])